# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
import csv
cities_df = "../starter_code/city_data_reports.csv"
cities_data_df = pd.read_csv(cities_df)
cities_data_df

,Unnamed: 0,City,City Latitude,City Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),City Country,City Datetime
0,0,albany,42.6001,-73.9662,51.89,55.58,88.0,76.0,2.51,US,1.634819e+09
1,1,buon me thuot,-15.8900,-52.2567,84.78,84.78,58.0,0.0,0.00,BR,1.634819e+09
2,2,barra do garcas,-46.6000,168.3333,43.41,43.41,81.0,1.0,9.24,NZ,1.634819e+09
3,3,bluff,40.2883,38.4236,56.70,56.70,30.0,0.0,2.53,TR,1.634819e+09
4,4,sebinkarahisar,51.8657,-2.2431,53.53,57.47,61.0,40.0,11.50,GB,1.634819e+09
...,...,...,...,...,...,...,...,...,...,...,...
577,583,nata,48.0056,91.6419,37.26,37.26,42.0,6.0,0.78,MN,1.634820e+09
578,584,caiaponia,35.5070,27.2132,69.76,69.76,56.0,20.0,20.71,GR,1.634820e+09
579,585,tarko-sale,36.6520,107.7896,45.18,45.18,40.0,0.0,4.79,CN,1.634820e+09
580,586,zelenoborskiy,28.6585,-17.9182,73.15,73.15,73.0,11.0,13.80,ES,1.634820e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = cities_data_df[["City Latitude", "City Longitude"]].astype(float)
humidity = cities_data_df["Humidity (%)"].astype(float)

In [7]:
fig = gmaps.figure(center = (-2.77, -42.03), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
cities_weather = cities_data_df.loc[(cities_data_df["Max Temperature (F)"]<90) & (cities_data_df["Max Temperature (F)"]>70)
                                       & (cities_data_df["Wind Speed (mph)"]<10) & (cities_data_df["Cloud Coverage (%)"]<20)
                                       & (cities_data_df['Humidity (%)']<80)]
cities_weather = cities_weather.drop(cities_weather.columns[0], axis = 1)

In [9]:
cities_weather.reset_index(inplace = False)
cities_weather = cities_weather.dropna()
cities_weather.reset_index(drop = True)

,City,City Latitude,City Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),City Country,City Datetime
0,buon me thuot,-15.8900,-52.2567,84.78,84.78,58.0,0.0,0.00,BR,1.634819e+09
1,punta arenas,19.7297,-155.0900,66.49,76.28,73.0,1.0,0.00,US,1.634819e+09
2,dikson,24.7667,82.7833,74.55,74.55,57.0,5.0,7.40,IN,1.634819e+09
3,pisco,30.9000,75.8500,85.26,85.26,17.0,0.0,5.53,IN,1.634819e+09
4,ludhiana,29.0331,21.5482,82.17,82.17,31.0,2.0,6.15,LY,1.634819e+09
5,chegdomyn,-19.6702,30.0059,81.82,81.82,20.0,11.0,8.30,ZW,1.634819e+09
6,lovington,-16.4833,-151.7500,77.79,77.79,79.0,17.0,7.58,PF,1.634819e+09
7,ashtabula,21.3906,-158.1547,72.99,73.71,73.0,1.0,0.00,US,1.634819e+09
8,nishihara,-20.1883,32.6236,82.71,82.71,26.0,2.0,2.01,ZW,1.634819e+09
9,henties bay,-25.2406,130.9889,78.64,78.64,17.0,0.0,6.91,AU,1.634819e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = cities_weather.reset_index(drop = True)
hotel_df['Hotel Name'] = ""
hotel_df

,City,City Latitude,City Longitude,Temperature (F),Max Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),City Country,City Datetime,Hotel Name
0,buon me thuot,-15.8900,-52.2567,84.78,84.78,58.0,0.0,0.00,BR,1.634819e+09,
1,punta arenas,19.7297,-155.0900,66.49,76.28,73.0,1.0,0.00,US,1.634819e+09,
2,dikson,24.7667,82.7833,74.55,74.55,57.0,5.0,7.40,IN,1.634819e+09,
3,pisco,30.9000,75.8500,85.26,85.26,17.0,0.0,5.53,IN,1.634819e+09,
4,ludhiana,29.0331,21.5482,82.17,82.17,31.0,2.0,6.15,LY,1.634819e+09,
5,chegdomyn,-19.6702,30.0059,81.82,81.82,20.0,11.0,8.30,ZW,1.634819e+09,
6,lovington,-16.4833,-151.7500,77.79,77.79,79.0,17.0,7.58,PF,1.634819e+09,
7,ashtabula,21.3906,-158.1547,72.99,73.71,73.0,1.0,0.00,US,1.634819e+09,
8,nishihara,-20.1883,32.6236,82.71,82.71,26.0,2.0,2.01,ZW,1.634819e+09,
9,henties bay,-25.2406,130.9889,78.64,78.64,17.0,0.0,6.91,AU,1.634819e+09,


In [11]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    #define variables for latitude and longitude
    lat = row["City Latitude"]
    lng = row["City Longitude"]
    
    params = {"location": f"{lat},{lng}", "rankby": "distance", "type": "lodging", "key": g_key,}
    city_name = row["City"]

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No hotel nearby... Skipping.")
        print("------------")

Retrieving Results for Index 0: buon me thuot.
Closest hotel to buon me thuot is Pousada Portal do Araguaia.
Retrieving Results for Index 1: punta arenas.
Closest hotel to punta arenas is Dolphin Bay Hotel.
Retrieving Results for Index 2: dikson.
Closest hotel to dikson is Abbas Ujjwal Tent House.
Retrieving Results for Index 3: pisco.
Closest hotel to pisco is RR STUDIO OPP WATER TANK.
Retrieving Results for Index 4: ludhiana.
Closest hotel to ludhiana is صاحبة الفخامة.
Retrieving Results for Index 5: chegdomyn.
Closest hotel to chegdomyn is Flame Lily Lodge.
Retrieving Results for Index 6: lovington.
Closest hotel to lovington is Teina Pascale.
Retrieving Results for Index 7: ashtabula.
Closest hotel to ashtabula is Polynesia Lodging & Conference.
Retrieving Results for Index 8: nishihara.
Closest hotel to nishihara is Chipinge Breeze Lodge.
Retrieving Results for Index 9: henties bay.
Closest hotel to henties bay is Ayers Rock Campground.
Retrieving Results for Index 10: maroantsetr

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>City Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Latitude", "City Longitude"]]

In [18]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

# Display figure
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))